# MarketBasket
### Overview
In market basket analysis, the **basket** refers to a customer's collection of items during a single purchase trip. It's not literally a physical basket, but rather the data that represents what products a customer buys together. The basket are represented as **sets** of indices, each index referring to a specific product (e.g. `{112, 41, 1020}` is a basket). 

By identifying frequently bought itemsets (groups of items purchased together), we can reveal associations between products. This allows retailers to understand which items customers tend to buy together.  This knowledge is crucial for strategies like targeted promotions.



In [ ]:
# First we need a dataset. 
# Let us generate a dataset in python
def random_basket(items, max_basket_size=15):
    basket_size = random.randint(1,max_basket_size)
    return {random.randint(0,items-1) for i in range(basket_size)}

import random
dataset_size, items = 100000, 100
dataset = [random_basket(items, 10) for i in range(dataset_size)]
print(dataset[:3])

In [ ]:
# Let's intialize the spark context and let's parallelize the data
import os

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Apriori").getOrCreate()

In [ ]:
# let's insert data in spark
rdd = spark.sparkContext.parallelize(dataset)

## The A-Priori Algorithm
This algorithm leverages a key property of itemsets - if a large itemset is frequent, all its smaller subsets must also be frequent. (e.g. if `{12,54,22,92,69,4}` is frequent then also all its subsets are frequent, therefore sets as `{12,54,22}` and `{12,69,4}` are frequent). Itemsets are considered frequent (or interesting) when their frequency exceeds a threshold parameterd---called **support**.

> The Role of Support:  The Apriori algorithm uses a minimum support threshold. This threshold defines how frequent an itemset needs to be considered "interesting" for further analysis.  Items or itemsets that appear less frequently than the threshold are discarded. Therefore, it is crucial to select a support that filters most of the data (to maintain the algorithm light) while not discarding interesting connections.

- The first step of the A-priori algorithm is to count occurencies of each item

In [ ]:
support = 310
first_pass = rdd.flatMap(lambda basket:[(e,1) for e in basket]) \
                .reduceByKey(lambda x,y:x+y) \
                .filter(lambda x:x[1]>support)

print("remaining singleton", first_pass.count())
print("5 random singleton", first_pass.take(5))

- Now we need to count all pair composed of frequent singletons

In [ ]:
frequent_singletons = set(first_pass.map(lambda x:x[0]).collect())
second_pass = rdd.flatMap(lambda basket:[((e0,e1),1) for e0 in basket for e1 in basket if e1 != e0 and e0 in frequent_singletons and e1 in frequent_singletons]) \
                 .reduceByKey(lambda x,y: x+y) \
                 .filter(lambda x:x[1]>support)
print(second_pass.count())

- Now we need to count all the triplets

In [ ]:
frequent_pairs = set(second_pass.map(lambda x:x[0]).collect())
third_pass = rdd.flatMap(lambda basket:[((e0,e1,2),1) for e0 in basket for e1 in basket for e2 in basket if \
                                         e1 != e0 and e0 != e2 and (e0,e1) in frequent_pairs and (e1,e2) in frequent_pairs and (e0,e2) in frequent_pairs]) \
                 .reduceByKey(lambda x,y: x+y) \
                 .filter(lambda x:x[1]>support)
print(third_pass.count())

You get the point, we simply reiterate this simple steps until we have no more frequent itemsets

In [ ]:
from itertools import combinations

support = 3

frdd = rdd.flatMap(lambda basket:[(e,1) for e in basket]) \
          .reduceByKey(lambda x,y:x+y) \
          .filter(lambda x:x[1] > support)
frequent = set(first_pass.map(lambda x:(x[0],)).collect())

print(f"remaining: {len(frequent)}, frdd {frdd.take(5)}")
    
k = 2
while frdd.count() != 0:
    frdd = rdd.flatMap(lambda basket: [(x,1) for x in combinations(basket,k) if all([y in frequent for y in combinations(x,len(x)-1)])]) \
              .reduceByKey(lambda x,y:x+y) \
              .filter(lambda x:x[1] > support)
    
    frequent = set(frdd.map(lambda x:x[0]).collect())
    print(k, len(frequent), frdd.take(5))
    k += 1

(⭐⭐⭐) repeat this algorithm with the data in `data.txt`

## PCY Algorithm
The PCY algorithm, also referred to as the Park-Chen-Yu algorithm, serves as a data mining technique specifically designed to detect frequently occurring itemsets within expansive datasets. It represents an enhancement over the Apriori algorithm.

In the Apriori algorithm, we would only tally itemsets if all their respective subsets were frequent. For instance, an itemset like `(i,j)` would only be counted if both `i` and `j` were frequent. With PCY, an additional discriminator is employed to ascertain the actual frequency of an itemset—**bucket**.

A bucket essentially functions as a counter linked to a set of itemsets. Let's denote these itemsets as $X=\{\text{itemset}_1,\dots,\text{itemset}_m\}$ associated with the same bucket. If these itemsets collectively appear a total of $k$ times, the bucket's counter will reflect this count. Consequently, it's evident that each itemset within $X$ cannot exceed a count of $k$ (as exceeding this count would result in a bucket count greater than $k$).

To illustrate, consider the itemset collection $X=\{\{11,3\},\{1,12\},\{13,2\},\{4,5\}\}$. Let's assume all itemsets in $X$ are linked to the same bucket. If the itemsets $\{11,3\},\{1,12\},\{13,2\},\{4,5\}$ respectively occur 1, 2, 5, and 2 times among the baskets, the associated bucket count would be $1+2+5+2=10$. Consequently, we deduce that no itemset in $X$ appears more than 10 times. Hence, if our chosen support threshold were $3$, we infer the presence of frequent itemsets within $X$. Conversely, if the support threshold were $15$, we conclude that none of the itemsets in $X$ are frequent.

We can link itemset to bucket by simply using hash functions.

In [ ]:
print(hash((1,2)))
# what if we want a bucket ?
print(hash((1,2))%10)

In the first step of the algorithm we will compute the counts for each singleton (same as in the Apriori algorithm) and we will also compute the bucket frequency for each pair.

In [ ]:
def from_buckets_to_bitmap(buckets, nobuckets):
    zero = [0] * nobuckets
    for b in buckets.collect(): zero[b[0]] = b[1]
    return zero

In [ ]:
support = 310
buckets = 10000
first_pass = rdd.flatMap(lambda basket:[(e,1) for e in basket]) \
                .reduceByKey(lambda x,y:x+y) \
                .filter(lambda x:x[1]>support)

first_buckets = rdd.flatMap(lambda basket:[(hash(e)%buckets,1) for e in combinations(basket,2)]) \
                .reduceByKey(lambda x,y:x+y) \
                .map(lambda x:(x[0],int(x[1] > support)))
first_bitmap = from_buckets_to_bitmap(first_buckets, buckets)


print("remaining singleton", first_pass.count())
print("5 random singleton", first_pass.take(5))
print(f"frequent buckets:{sum(first_bitmap)}, bitmap:", "".join(map(str,first_bitmap[:100])))

In the next step, we will compute the frequency of pairs for which
- the respective bucket is set to `1`.
- their singleton are all frequent (exceed the support)

Additionally, we will also compute buckets for the triplets 

In [ ]:
frequent_singletons = set(first_pass.map(lambda x:x[0]).collect())

second_pass = rdd.flatMap(lambda basket:[(e,1) for e in combinations(basket,2) if  
                                         e[0] in frequent_singletons and 
                                         e[1] in frequent_singletons]) \
                 .filter(lambda x:bitmap[hash(x[0])%buckets]) \
                 .reduceByKey(lambda x,y: x+y) \
                 .filter(lambda x:x[1]>support)

second_buckets = rdd.flatMap(lambda basket:[(hash(e)%buckets,1) for e in combinations(basket,3)]) \
                .reduceByKey(lambda x,y:x+y) \
                .map(lambda x:(x[0],int(x[1] > support)))

second_bitmap = from_buckets_to_bitmap(second_buckets, buckets)

print(second_pass.take(5))
print(f"frequent buckets:{sum(second_bitmap)}, bitmap:", "".join(map(str,second_bitmap[:100])))

In the next step, we need to compute the frequency of triplets for which
- the respective bucket is set to `1`.
- their pars are all frequent (exceed the support)

Additionally, we will need also compute buckets for the quadruplets. And so on.

Let us instead focus on the iterative algorithm.

In [ ]:
support = 3
buckets = 1000000

k = 1
# lets compute the frequent singletons
frdd = rdd.flatMap(lambda basket:[(e,1) for e in basket]) \
          .reduceByKey(lambda x,y:x+y) \
          .filter(lambda x:x[1] > support)

# lets compute bucket counters for all the pairs in the baskets
fbuckets = rdd.flatMap(lambda basket:[(hash(e)%buckets,1) for e in combinations(basket,2)]) \
            .reduceByKey(lambda x,y:x+y) \
            .map(lambda x:(x[0],int(x[1] > support)))

# from the buckets counters, we get the bitmap (1 for frequent bucket)
bitmap = from_buckets_to_bitmap(fbuckets, buckets)

# the frequent elements (same as the Apriori)
frequent = set(first_pass.map(lambda x:(x[0],)).collect())

print(f"it:{k}, frequents:{len(frequent)}, bitmap:{sum(bitmap)}, samples:{frdd.take(3)}")

k = 2
while frdd.count() != 0:

    # we use the frequent elements and the bitmap to filter the itemsets
    frdd = (rdd.flatMap(lambda basket: [(x,1) for x in combinations(basket,k)]) # compute all itemsets of size k from each basket
              .filter(lambda x: all([y in frequent for y in combinations(x[0],len(x[0])-1)])) # filter itemsets that have a non-frequent sub-itemset
              .filter(lambda x:bitmap[hash(x[0])%buckets]) # filter itemsets with 0 bitamp value
              .reduceByKey(lambda x,y:x+y) # count remaining itemsets
              .filter(lambda x:x[1] > support)) # filter those that are frequent

    # now we compute the buckets and the bitmap for the next iteration
    fbuckets = (rdd.flatMap(lambda basket:[(hash(e)%buckets,1) for e in combinations(basket,k+1)]) # hash each itemset on its bucket 
            .reduceByKey(lambda x,y:x+y) # count buckets
            .map(lambda x:(x[0],int(x[1] > support)))) # map to 1 those that are frequent 
    bitmap = from_buckets_to_bitmap(fbuckets, buckets) # compute the bitmap

    # get the frequent elements for the next step
    frequent = set(frdd.map(lambda x:x[0]).collect())
    
    print(f"it:{k}, frequents:{len(frequent)}, bitmap:{sum(bitmap)}, samples:{frdd.take(3)}")
    k += 1

(⭐⭐⭐) repeat this algorithm with the data in `data.txt`